딥러닝에서 가장 주요한 문제 중 하나는 Vanishing/Exploding 그라디언트 문제이다. <br />
레이어의 갯수가 적은 경우에서는 그리 큰 문제가 되지 않지만, 레이어의 갯수가 많아지면 많아질수록 누적되어 나타나기 때문에 문제가 심각해진다. <br />
그 이유는 활성화 함수로 sigmoid나 tanh와 같은 비선형 포화함수(non linear saturating function)을 사용하게 되면, 입력의 절대값이 작은 일부 구간을 <br />
제외하면 미분값이 0 근처로 가기 때문에 back propagation을 통한 학습이 어려워지거나 느려지게 된다.

이 문제에 대한 해결책으로 2011년 ReLU(Reticifier Linear Unit)을 활성화 함수로 쓰는 방법이 소개되어 문제가 어느정도 완화되기는 하였지만, 이것은 <br />
간접적인 해결 방법이지 본질적인 해결책은 아니기 때문에 네트워크가 깊어질수록 여전히 문제가 된다. <br />
Drop out이나 기타 Regularization 방법들 역시 본질적인 해결책은 아니기 때문에 여전히 일정 레이어의 갯수를 넘어가게 되면 traing을 성공시킨다는 것을 보장할 수 없게 된다. 

그러다가 2015년에 획기적인 방법 두 개가 발표되는데, BN(Batch Normalization)과 Residual Network이다.

BN은 2015년 발표된 이래 최근 딥러닝에는 거의 대부분 적용이 되는 추세이며, ResNet에도 BN이 적용되었고, 구글의 Inception V2 이후의 후속 구조에는 모두 BN이 적용되었다.<br />
BN은 구글의 Sergey loffe와 Christian Szegedy가 공동으로 발표하였으며, <br />
"Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift"를 중심으로 자세히 알아보자.

## Internal Covariate Shift

최근 딥러닝에는 대부분 GPU가 사용되고 있으며, GPU를 효율적으로 사용할 수 있도록 보통은 32~256 크기를 갖는 mini-batch SGD(Stochastic gradient descent)<br />
방법을 많이 사용한다. <br />
SGD 방식이 효율적이기는 하지만, 효과를 거두려면 하이퍼 파라미터의 설정에 주의하여야 하며, 특히 초기값과 학습 진도율(Learning rate)은 매우 중요가 된다. <br />
학습 시에 현재 레이어의 입력은 모든 이전 레이어의 파라미터의 변화에 영향을 받게 되며, 네트워크가 깊어짐에 따라 이전 레이어의 작은 파라미터 변화가 증폭되어 <br />
뒷 단에 큰 영향을 끼칠 수 있다. <br />
이처럼, 학습하는 도중에 이전 레이어의 파라미터 변화로 인해 현재 레이어 입력의 분포가 바뀌는 현상을 "Covariate Shift"라고 한다.

이는 마치 Quora에서 Debiparad Ghosh가 설명한 것처럼, Covariate Shift는 건축에서 하중에 의해 기둥이 휘어지는 Buckling과 비슷하다고 볼 수 있다.
___

<img src="./Images/1.png" width=600 />

기둥이 휘어지는 것을 막을면 위 그림에서 (c)나 (d)의 경우처럼 휘어짐을 방지하는 수단이 필요하게 되며, 그 수단으로는 Batch Normaliazation이나 <br />
Whitening 기법을 들 수 있다.
___

## Covariate Shift를 줄이는 방법

Internal Covariate Shift를 줄이는 대표적인 방법 중 하나는 각 레이어로 들어가는 입력을 Whitening 시키는 것이다. 여기서 Whitening을 시킨다는 의미는 <br />
평균 0, 분산 1로 바꾸어 준다는 말이다. <br />
하지만 단순하게 Whitening만을 시킨다면, Whitening 과정과 파라미터를 계산하기 위한 최적화 과정(Back propagation)은 서로 무관하게 진행이 되기 때문에, <br />
특정 파라미터가 계속 커지는 상태로 Whitening이 진행될 수 있다. <br />
논문이 나오는 것처럼 Whitening을 통해 loss(cost function)이 변하지 않게 되면, 최적화 과정을 거치면서 특정 변수가 계속 커지는 현상이 발생할 수 있다. <br />
그러므로 단순하게 Whitening을 통해 평균과 분산을 조정하는 것보다는 좀 더 확실한 방법이 필요하며, 그것이 바로 BN(Batch Normalization)이다.

BN은 평균과 분산을 조정하는 과정이 별도의 프로세스로 있는 것이 아니라, 뉴럴 네트워크 안에 포함이 되어 학습 시에 평균과 분산을 조정하는 과정 역시 같이 조절이 된다는 점이 <br />
단순 Whitening과 구별되는 점이다.

## Batch Normalization(BN)

Normalization은 원래 training 전체 집합에 대하여 실시하는 것이 최고의 효과를 거둘 수 있겠지만, mini batch SGD 방식을 사용하게 되면, 파라미터의 <br />
업데이터가 mini batch 단위로 일어나기 때문에, mini batch 단위로 실시한다. 단 mini batch 집합의 선정은 가급적이면 correlation이 적어 mini batch가 <br />
전체 집합을 대표하는 것이라고 생각해도 무방하도록 해주어야 한다. <br />
학습 시 BN 방법은 아래와 같다.
___

<img src="./Images/2.png" width=500 />

평균과 분산을 구하게 되면 입력을 정규화(normalize) 시킨다. 정규화(normalize)과정에서 평균을 빼주고 그것을 분산으로 나누어주게 되면 분포는 -1 ~ 1의 범위가 된다.
___

BN이 whitening과 다른 부분은 평균과 분산을 구한 후 정규화(normalize)시키고 다시 scale과 shift 연산을 위해 $\gamma$와 $\beta$가 추가되었으며, <br />
$\gamma$와 $\beta$가 추가됨으로써 정규화(normalize)시켰던 부분을 원래대로 돌리는 identity mapping도 가능하고, 학습을 통해 $\gamma$와 $\beta$를 정할 수 있기 때문에 <br /> 
단순하게 정규화(normalize)만을 할 때 보다 훨씬 강력해진다.

BN은 보통 non linear 활성화 함수 앞쪽에 배치가 되며 아래의 그림과 같은 형태가 된다.
___

<img src="./Images/3.png" width=600 />

BN은 뉴럴 네트워크에 포함이 되기 때문에 back propagation을 통해 학습이 가능하다.
___

back propagation 시에는 아래와 같은 Chain rule이 적용된다.
___

<img src="./Images/4.png" width=600 />
___

## Training과 Test의 차이

BN은 테스트 할 때와 학습을 할 때 적용하는 방법이 조금 다르다. <br />
학습 시에는 각 mini batch마다 $\gamma$와 $\beta$를 구하고 그 값을 저장해 놓는다. <br />
Test 시에는 학습 시 mini batch마다 구했던 $\gamma$와 $\beta$의 평균을 사용한다는 점이 다르며, 아래의 그림처럼 표현이 가능하다.
___

<img src="./Images/5.png" width=600 />
___

테스트 시의 Pseudo code는 아래와 같다.
___

<img src="./Images/6.png" width=600 />

유사 코드를 보면 알 수 있듯이 평균은 각 mini batch에서 구한 평균들의 평균을 사용하고, 분산은 분산의 평균에 $\frac {m}{m-1}$을 곱해주는 점이 다르다.
___

여기서 $\frac {m}{m-1}$를 곱해주는 이유는 통계학적으로 unbiased variance에는 "Bessel's correction"을 통해 보정을 해주는 것이다. <br />
이는 학습 전체 데이터에 대한 분산이 아니라 mini batch들의 분산을 통해 전체 분산을 추정할 때 통계학적으로 보정을 위해 베셀의 보정값을 곱해주는 방식으로 추정하기 때문이다.

종합하면, Batch Normalization은 단순하게 평균과 분산을 구하는 것이 아니라, scale($\gamma$)와 shift($\beta$)를 통한 변환을 통해 훨씬 유용하게 되었으며, <br />
BN이 뉴럴 네트워크의 레이어 중간에 위치하게 되어 학습을 통해 $\gamma$와 $\beta$를 구할 수 있게 되었다. <br />
Covariate shift 문제로 인해 네트워크가 깊어질 경우 학습에 많은 어려움이 있었지만, BN을 통해 Covariate shift 문제를 줄여줌으로써 학습의 결과도 좋아지고 빨라지게 된다.

## BN을 convolution network에 적용하는 방법

BN은 활성화 함수의 종류와 상관없이 적용 가능하다. <br />
활성화 함수를 $g$라 하고 가중치를 $W$, 입력 데이터를 $u$, 바이어스를 $b$라고 하면, 다음은 기본 뉴럴 네트워크를 표현하는 함수 형태가 된다. <br />
$z=g(Wu+b)$ <br />

Non linearity 함수 $g$앞에 BN을 적용하게 되면, 결과적으로 $x=Wu+b$를 정규화(normalization)하는 것이 된다. <br />
$x=Wu+b$에 BN을 적용하면, 정규화(normalization) 후 scale과 shift 항을 학습을 통해 결정하는데, $b$는 shift 항으로 대체할 수 있기 때문에 무시 할 수 있으며, <br />
앞선 식을 다시 정리하면 아래와 같은 형태가 된다. <br />
$z=g(BN(Wu))$ <br />
여기서 BN transform은 각각의 activation에 $x=Wu$에 독립적으로 적용하며, 또한 학습을 통해 변수 $\gamma$와 $\beta$가 쌍으로 결정이 된다.

하지만 Conv 레이어에 적용할 때는 조금 달라지는 부분이 있는데, 이는 convolution의 특성을 살리기 위함이다. <br />
Conv 레이어에서는 shared weight와 sliding window 방식을 적용하여 출력 feature map의 모든 픽셀에 대하여 동일한 연산을 수행한다. <br />
마찬가지로 BN을 적용할 때는 mini batch에 있는 모든 activation 뿐만 아니라 모든 위치까지 함께 고려해 주어야 한다. 

BN을 Conv 레이어에 적용할 때 mini batch의 크기를 $m$이라고 하고, 출력 feature map의 크기를 $p$x$q$라고 하면, 출력 feature map의 크기에 <br />
해당하는 만큼 sliding window를 움직이면서 연산을 해주기 때문에 mini batch의 크기가 사실상 $m$`=$m$x$p$x$q$로 커진 것으로 간주하고 평균과 분산을 구한다. <br />
$\gamma$와 $\beta$는 fully connected의 경우처럼 activation마다 붙는 것이 아니라, feature map에 대하여 $\gamma$와 $\beta$쌍이 학습을 통해 결정이 된다.

## MNIST 데이터를 이용한 실험

BN이 과연 효과가 있는지를 확인하기 위해 MNIST 데이터 셋을 이용한 실험을 하였다. <br />
MNIST 데이터 셋은 필기체 숫자 인식을 위한 데이터 집합이며 6만장의 학습 이미지와 1만장의 테스트 이미지가 있다. <br />
Yann LeCun 등이 필기체 인식에 사용하였으며, 데이터 집합의 크기가 작아 CNN 알고리즘의 초기 검증에 많이 사용한다. <br />
MNIST 데이터의 예는 다음 그림과 같다.
___

<img src="./Images/7.png" width=500 />

MNIST 데이터를 검증에 사용한 이유는 FC 레이어로만 구성된 간단한 뉴럴 네트워크를 구성하고 과연 BN을 적용했을 때 효과가 있는지를 확인하기 위함이다.
___

MNIST 데이터 실험에는 font의 크기가 28x28이므로 총 784개의 입력을 받아 100개의 뉴런(actication)을 가진 FC hidden 레이어 3개 및 10개의 출력으로 구성되는 <br /> 
간단한 네트워크를 구현하였다.

이 간단한 네트워크에 대하여 BN을 적용했을 때와 그렇지 않을 경우에 대한 시험을 하였는데 결과는 아래의 그림과 같다.
___

<img src="./Images/8.png" width=700 />

그림 (a)를 보면 알 수 있듯이, BN을 적용한 경우에 학습 속도가 훨씬 더 빠르고 학습 결과도 좋다. <br />
그림 (b)와 그림 (c)는 마지막 hidden 레이어에 있는 특정 뉴런(activation)에서 입력 분포 값을 보여주는 것이다. <br />
BN이 적용된 경우는 거의 변화가 없이 안정적이지만 BN이 적용되지 않는 경우는 값이 흔들리는 것을 확인할 수 있다.
___

## Google Inception 구조에 ImageNet 데이터를 이용한 실험

BN이 FC 레이어로 구성된 간단한 네트워크 뿐만 아니라, 실제로 복잡한 Deep CNN에서도 효과가 있는지 실험을 하였다. <br />
이미 2014년 구글 팀은 Inception 구조를 이용하여 ILSVRC 대회에서 우승을 하였다. <br />
그 Inception 구조에 BN을 적용하였을 때 효과적인지를 입증할 수 있다면 BN의 타당성을 입증할 수 있게 된다.

## 기본 인셉션 구조의 변형

기존 Inception 구조에 그대로 BN을 적용하면 기대했던 효과를 거둘 수 없기 때문에 BN의 특성에 맞게 기존 Inception 구조를 약간 변형시켰다. <br />
- 학습 진도율(Learning rate)의 값을 향상 => 속도 향상 <br />
- Drop out 제거 => BN을 적용하면 regularization 효과를 얻을 수 있기 때문에 drop out 레이어를 제거 가능. <br />
- 학습 데이터로부터 mini batch를 정할 때 좀 더 철저하게 섞기 => correlation이 적을수록 효과적이기 때문에 같은 데이터가 mini batch에 포함이 되지 않도록 했으며, <br />
이를 통해 약 1%정도의 validation 정밀도가 향상됨.
- ${L}_{2}$ weight regularization 값을 줄이기 => ${L}_{2}$ weight를 1/5로 줄였더니 효과적이었다.
- 학습 진도율(Learning rates)를 더 빨리 감소시키기 => BN을 적용하면 학습 속도가 빠르기 때문에 기존 인셉션때 보다 Learning rates를 줄여주는 것을 더 빠르게 적용하였다. <br /> 
(6배)
- Local Response Normalization 제거 => LRN은 거의 제거하는 것이 대세이다.
- Photometric distortion을 줄이기 => 학습 시간이 빠르기 때문에 인위적인 변경을 통해 학습 데이터를 늘리는 대신에 real data 테스트에 집중. <br />
(변경을 전혀 시키지 않는 것은 아니고 변경의 강도를 줄임)

## 실험에 사용한 다양한 구조

다양한 구조 실험을 통해 어떤 구조가 효과적인지를 확인할 수 있도록 다음과 같은 5개의 구조를 실험에 사용하였다. <br />
- Inception: 원래 인셉션 구조. 초기 Learning rates로 0.0015를 사용.
- BN-Baseline: 기본 인셉션과 동일하지만 각 non linearity 앞에 BN을 배치.
- BN-x5: BN-Baseline에서 초기 Learning rates를 5배 키워 0.0075 적용.
- BN-x30: BN-x5와 비슷하나 초기 Learning rates를 30배 키워 0.045 적용.
- BN-x5-Sigmoid: BN-x5와 동일하지만, non linearity에 ReLU 대신 Sigmoid를 사용.

위 5개 구조에 대한 실험 결과는 다음과 같다.
___

<img src="./Images/9.png" width=500 />

위 결과는 구조 비교를 위해 single model - single crop에 대한 실험을 하였으며, 기존 Inception의 경우 31x${10}^{6}$만큼 학습을 해주어야 정확도가 72.2%가 되었다.
___

기본 Inception 구조에 BN만 추가한 BN-Baseline의 경우, 기존 Inception보다 절반 정도의 학습 과정만을 거쳐도 비슷한 수준의 정확도를 얻을 수 있었다. <br />
Learning rates를 키운 BN-x5의 경우는 기존 Inception보다 14배 빠르게 비슷한 정확도에 도달할 수 있었다. <br />
Learning rates를 더 키운 BN-x30은 비슷한 정확도에 도달하는데 기존 Inception보다 6배 빠른 수준이고, 정확도는 74.8%까지 올라가 정확도가 개선되는 것을 확인 할 수 있었다. <br />
ReLU 대신에 Sigmoid 함수를 사용하는 경우에도 BN을 사용하면 학습을 어느정도 성공시킬 수 있었으며 69.8%까지 얻을 수 있었다. <br />
종합하면 아래 표는 정확도 72.2%까지 도달하는데 걸리는 시간을 표시한 것이며, 각각 얻은 최종 정확도 역시 표시되어있다.
___

<img src="./Images/10.png" width=500 />

## 모델 결합을 통한 성능 비교(Ensemble Classification)

Sergey loffe가 논문을 발표할 당시만 하더라도 마이크로소프트의 ResNet이 최종결과를 보고하지 못하였다. <br />
Inception에 앞서 살펴본 방식과 같이 BN을 적용하고 multi crop, multi model에 대한 ensemble을 적용하였을 때, <br />
결과가 가장 좋은 것처럼 발표하였으나, 이는 곧 ResNet 최종 결과가 발표되면서 결과가 뒤집어진다. <br />
이후 다시 Inception-V3를 발표하여 자신들도 성능이 나온다고 주장을 하고 있다.
___

<img src="./Images/11.png" width=500 />
___

종합하면, Batch Normalization을 적용하면, Internal Covariate Shift 문제를 해결할 수 있으며, 이를 통해 딥러닝의 고질적인 문제들이 해결가능하다. <br />
Vanishing/Exploding 그라디언트 문제의 해결책이 생기면서 낮은 Learning rates가 아니라 높은 Learning rates를 적용할 수 있게 되어 학습 속도의 향상은 <br /> 
물론 학습의 정확도까지 개선할 수 있게 되었다.